In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-10-31 15:01:32--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?9CxsC431pfklbz2UZK6pjWGSi-5g-cT1uFAlChj4aXOPQVsosQtMPPP3NglNJiOSom5C5fGnC29hHNxbr4xbtMG6ciJ8Z8Oq5U_VZKKdCBcASXw2tP3U1uLr6sJd15db4B-oGjRacfTm-Adw2DLDk--ybzZosuzgAV2M4rpEobAR2KgIk14okmCHT3mJBnN2c3UE2wbX_1G1x-zdH7I [following]
--2020-10-31 15:01:32--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?9CxsC431pfklbz2UZK6pjWGSi-5g-cT1uFAlChj4aXOPQVsosQtMPPP3NglNJiOSom5C5fGnC29hHNxbr4xbtMG6ciJ8Z8Oq5U_VZKKdCBcASXw2tP3U1uLr6sJd

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wx4z0jem
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wx4z0jem
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=fa9764d1b904ef08529c76e620cfe093f6b4707f6d87d409b3f0429cc3ba990c
  Stored in directory: /tmp/pip-ephem-wheel-cache-baqkry_w/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [6]:
!nvidia-smi

Sat Oct 31 15:06:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
%%cu
#include <stdlib.h>
#include <stdio.h>
#include <fstream>
#include <cuda.h>
#include <iostream>
#include <iomanip>
#include <time.h>

using namespace std;
#define TILE 16


//Initialize a 2D matrix   
void initialize_matrices(double** a, double** l, double** u, int size)
{
  //for each row in the 2d array, initialize the values
  for (int i = 0; i < size; ++i)
  {
    a[i] = new double[size];
    l[i] = new double[size];
    u[i] = new double[size];
  }
}
//~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
// Scale the index for threads to get pivot starting and ending points
//~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
__global__ void scaleIndex(double *matrix, int n, int index){
  int start=(index*n+index);
	int end=(index*n+n);
	
	for(int i= start+1 ; i<end; ++i){
		matrix[i]=(matrix[i]/matrix[start]);   //This step is used for calculating each element of U matrix
	}

}

//~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
// Row elimination Kernel - takes matrix, dimension, currect row index, and block size
//~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
__global__ void elim(double *A, int n, int index, int bsize){
	extern __shared__ double pivot[];

	int idThread=threadIdx.x;
	int idBlock=blockIdx.x;
	int blockSize=bsize;


	if(idThread==0){
	     for(int i=index;i<n;i++) pivot[i]=A[(index*n)+i];
	}

	__syncthreads();
  //Variables for pivot, current-row, start and end
	int pivotRow=(index*n);
	int currentRow=(((blockSize*idBlock) + idThread)*n);
	int start=currentRow+index;
	int end=currentRow+n;
  //If greater than pivot row, loop from start index + 1(next row) to end of column
	if(currentRow >pivotRow){
    for(int i= start+1; i<end; ++i){
        //Set the matrix value of next row and its column - pivot
        A[i]=A[i]-(A[start]*pivot[i-currentRow]);

             }
      }
}
//Randomly generated diagonal dominant (non-singular) matrix - 1D
void fillMatrix(double* a, int n){
  // Fill the matrix
   for (int i = 0; i <= (n*n); ++i) {
    a[i] =((rand()%10)+1);
  }

  //Make the matrix diagonally dominant to guarantee it is non-singular (invertible)
  int diagCount = 0;
  double sum = 0;
  for(int i = 0; i < n; ++i){
    //Iterate through the row, add all the values, remove the diagonal value
    for(int j = i*n; j < i*n + n; ++j){
      sum += abs(a[j]);
      //printf("%f +", sum);
    }
    ///Remove the diagonal value
    //i*n gives us the current row, then  add diagCount to get to correct column
    sum -= abs(a[i*n + diagCount]);
    //Add random value to the new sum, this guarantees diagonal is now larger than row sum
    a[i*n + diagCount] = sum + ((rand()%5)+1);
    ++diagCount;
    sum = 0;
  }

}

//----------------------------------------------------------------------- 
//Store 1D Matrix
//~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
void printMatrix(double* a, int n, FILE* Output){
    for(int i=0; i<(n*n); ++i){
           if(i%n==0)
       		   //cout << endl << left << setw(9) << setprecision(3) << a[i] << left <<  setw(9);
              fprintf(Output,"\n%4.4f ",a[i]);
           else 
            //cout << left << setw(9) << setprecision(3) << a[i] << left <<  setw(9);
            fprintf(Output,"%4.4f ",a[i]);
         }
    //printf("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n");
    cout << endl;
}
//----------------------------------------------------------------------- 
//Store 2D Matrix
//----------------------------------------------------------------------- 
void print2D(double** matrix, int size,FILE* Output)
{
  //for each row...
  for (int i = 0; i < size; i++)
  {
    //for each column
    for (int j = 0; j < size; j++)
    {
      //print out the cell
      //cout << left << setw(9) << setprecision(3) << matrix[i][j] << left <<  setw(9);
     fprintf(Output,"%4.4f ",matrix[i][j]);
    }
    //new line when ever row is done
    //cout << endl;
   fprintf(Output,"\n");
  }
}

int main(int argc, char** argv){
  //Matrix dimension will be n*n
  int n=10;
  FILE* f;
  f = fopen("result_parallel.txt","r");
  char ch;
  long length;
  fseek(f, 0, SEEK_END);
  length = ftell(f);
  fseek(f, (length - 1), SEEK_SET);
  ch=fgetc(f);
  int count = ch-'0';
   fclose(f);
  FILE *Output;
  Output = fopen("result_parallel.txt","a+");
  srand(1);

  //Allocate A matrix, U, and L  for CPU
  double *a = new double[n*n];
  double *ret=new double[n*n];

  //Fill in diagonal dominant matrix, then print
  fillMatrix(a, n);
 

  //Allocate GPU memory for A matrix, get number of blocks
  double *da;
  int numblock = n/TILE + ((n%TILE)?1:0);

  double runtime;
  runtime = clock()/(double)CLOCKS_PER_SEC;
  cudaMalloc(&da, n*n* sizeof (double));
  cudaMemcpy(da, a, n*n*sizeof(double), cudaMemcpyHostToDevice);
  for(int i=0; i<n; ++i){
      scaleIndex<<<1,1>>>(da,n,i);
	    elim<<<numblock,TILE,n*sizeof(double)>>>(da,n,i,TILE);
   }
  cudaMemcpy(ret, da, n*n*sizeof(double),cudaMemcpyDeviceToHost );
  //printf("Returned Matrix\n");
  //printMatrix(ret,n);
  runtime = clock() - runtime;
  printf("For %u x %u Matrix\n",n,n);
  cout << "Runtime for LU Decomposition is: " << (runtime)/float(CLOCKS_PER_SEC) << endl;
  //printf("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n");
  //Create 2D matrices for the L and U
  double** A = new double* [n];
  double** u = new double* [n];
  double** l = new double* [n];
  double* X = new double[n];
  double* Y = new double[n];
  double* B = new double[n];
  //Initialize these matrices, and transfer ret values into A
  initialize_matrices(A,u,l,n);
  for(int i = 0 ;i < n ; ++i){
    for(int j= 0; j < n; ++j){
      A[i][j]=ret[i*n+j];
    }
  }
  //Take values diagonal values from returned array and pull L and U
  for(int i=0; i<n; i++){
    for(int j=0; j<n; j++){
        //Find diagonals
        for(int k=0; k<n; k++){
            //If the outermost for loop is larger or equal to k, then grab L values
            if(i>=k)
              l[i][k] = A[i][k];
            //Else the rest of the array is zeroes
            else l[i][k] = 0;
            //If loops at diagonal then enter 1 for U, if j > k then we're on upper part 
            //of Matrix so fill in values, 
            if(k==j)u[k][j] = 1;
            else if(k<j)u[k][j] = A[k][j];
            else u[k][j] = 0.0;
        }
    }
  }
  for (int i = 0; i < n; i++) {
    B[i] =((rand()%10)+1);
  }
  
  fprintf(Output,"\nMatrix A is:");
  //printf("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n");
  //printf("Matrix 'A' is:");
  printMatrix(a,n,Output); 
  fprintf(Output,"\nMatrix B is:\n");
  //printf("Matrix 'B' is:\n");
  for(int i=0;i<n;i++)
  {
     //cout << left << setw(9) << setprecision(3) << B[i] << left <<  setw(9);
      fprintf(Output,"%4.4f ",B[i]);
  }
  fprintf(Output,"\n");
  fprintf(Output,"Matrix L is:\n"); 
  //printf("Matrix 'L' is:\n");
  print2D(l,n,Output);
  //printf("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n");
  //printf("Matrix 'U' is: \n");
  fprintf(Output,"Matrix U is: \n");
  print2D(u,n,Output);
  //printf("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n");
  for(int i=0;i<n;i++)
  {
    Y[i]=B[i];
    for(int j=0;j<i;j++)
    {
      Y[i]-=l[i][j]*Y[j];
    }
   Y[i]/=l[i][i];
  }
  for(int i=n-1;i>=0;i--)
  {
    X[i]=Y[i];
    for(int j=i+1;j<n;j++)
    {
      X[i]-=u[i][j]*X[j];
    }
  }
  fprintf(Output,"Solution(X) is:\n");
  //printf("Solution(X) is:\n");
  for(int i=0;i<n;i++)
  {
     //cout << left << setw(9) << setprecision(3) << X[i] << left <<  setw(9);
      fprintf(Output,"%4.4f ",X[i]);
  }
  fprintf(Output,"\nFor %u x %u Matrix, Time taken is:",n,n);
  fprintf(Output,"%f\n\n",(runtime)/float(CLOCKS_PER_SEC));
  fprintf(Output,"Count %d",count+1);
  fclose(Output);
  cudaFree(da);
  cudaFree(ret);
  delete[] a;
  delete[] ret; 

  return 0;
}

For 10 x 10 Matrix
Runtime for LU Decomposition is: 0.204483


